In [90]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np 

In [92]:
def read(his_file,sim_num,fname): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 

# Remember all arrays are (time,kmax,jmax,imax) contrary to Matlab     
    imax=size(x[1,:])
    jmax=size(y[:,1])
     
# read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]

#Get Stretching function 
    N=10 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(11)
    Csur=np.zeros(11)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
        
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
    
# calculate the 4D field of z values (vertical coordinate)
    z0=(depth_c*s+depth*Csur)/(depth_c+depth)  
    
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,size(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1

# Find water level for all locations at last tidal cycle 
    z=np.zeros(shape=(t_slice,Np,jmax,imax))
    for t in range(t_tidal_srt,t_tidal_end+1): 
        eta = nc.variables['zeta'][t,:,:] 
        z[t-t_tidal_srt,:,:,:] = eta+(eta+depth)*z0
    
# Calculate maximum water level change in a tidal cycle before and after the patch
# Probe the region infront of the patch by adding 5 points in vertical 
# Probe the region behind the patch by adding 5 points in vertical 
# Water level change in the middle transect before and after the patch

    grad_wl=0.0
    for t in range(0,t_slice):
        for k in range(0,Np):
            grad=abs(z[t,k,jmin_plant-5,imid]-z[t,k,jmax_plant+5,imid])
            grad_wl=max(grad_wl,grad)
    
    print "Maximum gradient water level"
    print grad_wl 
    
    f.write("%s\t, %s\t \n" % (sim_num, grad_wl))


In [93]:
 '''Output from vegetation module COAWST 
Mean gradient of water level around vegetation p)[mts.]
'''    
fname='grad_water_level_output.txt'
f = open(fname, 'w')
total_sim_list=16
f.write("Case_no, Gradient of water level(mts.): \n")

for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/'
    path2='veg_test_his_'+ str(sim_num)+'.nc'
    url=path1+path2
    print url 
    read(url,sim_num,fname) 

/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_1.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0906050857157
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_2.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0776969827712
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_3.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0864368518814
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_4.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0932615157217
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_5.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0892242211848
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_6.nc
Last tidal cycle starts and ends at this instance
186 258
Maximum gradient water level
0.0767011023127
/media/gad